In [ ]:
run_env = 'on_cloud'
# 'on_cloud' / 'on_premise'

In [1]:
#Importing spark session and spark configurations
from pyspark.sql import SparkSession
def _get_spark():
  spark = SparkSession.builder.appName("project_customer_360_ops_report").getOrCreate()
  spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
  spark.conf.set("spark.sql.parquet.binaryAsString", "true")
  spark.conf.set("spark.sql.shuffle.partitions", 200)
  spark.conf.set("spark.sql.files.maxPartitionBytes", 1024*1024*256)
  return spark
spark = _get_spark()
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

In [2]:
#importing required spark functions
from pyspark.sql import functions as f
from pyspark.sql.functions import expr, col

In [ ]:
#Reading the latest metadata table and filtering the unwanted datasets /layers
if run_env == 'on_cloud'
    mtdt_tbl = spark.read.parquet('/mnt/customer360-blob-output/C360/UTILITIES/metadata_table')
else:
    mtdt_tbl = spark.read.parquet('/projects/prod/c360/data/UTILITIES/metadata_table')
ops_report = mtdt_tbl.where("table_name not like '%l4%'")
ops_report = ops_report.where("table_name not in ('int_l0_streaming_vimmi_table')")

In [3]:
#Finding the latest record of every dataset based on updated_on and target_max_data_load_date column.
ops_report = ops_report.withColumn("rn", expr("row_number() over (partition by table_name order by updated_on desc,target_max_data_load_date desc)") )
ops_report = ops_report.filter("rn = 1").drop("rn")

In [4]:
#Finding the latest record of every dataset based on updated_on and target_max_data_load_date column.
ops_report = ops_report.withColumn("rn", expr("row_number() over (partition by table_name order by updated_on desc,target_max_data_load_date desc)") )
ops_report = ops_report.filter("rn = 1").drop("rn")

In [5]:
#Calculate the Domain name and Feature layer from dataset path
ops_report = ops_report.withColumn("Domain_Name", f.split(f.col("table_path"),'/')[4]) \
                      .withColumn("Feature_Layer",f.split(f.col("table_path"),'/')[5]) \
                      .withColumnRenamed("table_name", "Dataset_Name") \
                      .withColumnRenamed("updated_on", "Last_Data_Refresh_Date")

In [6]:
#Calculate all the relevant columns of ops report.
ops_report = ops_report.withColumn("Days_Since_Last_Refresh", f.datediff(f.current_date(), f.col("Last_Data_Refresh_Date") ) ) \
                       .withColumnRenamed("target_max_data_load_date", "Latest_Data_Partition_Available") \
                       .withColumn("Data_Refresh_Freq", f.when( f.lower(f.col("Feature_Layer")) == 'l1_features', "daily").when( f.lower(f.col("Feature_Layer")) == 'l2_features', "weekly").when(f.lower(f.col("Feature_Layer")) == 'l3_features', "monthly").otherwise("No_frequency_defined") ) \
                        .withColumn("Data_Latency", f.when( (f.col("Data_Refresh_Freq") == 'daily'),f.datediff(f.current_date(), f.col("Latest_Data_Partition_Available")) ).when( (f.col("Data_Refresh_Freq") == 'weekly'), f.datediff(f.current_date(), f.date_sub(f.date_add(f.col("Latest_Data_Partition_Available"),7),1)) ).when( (f.col("Data_Refresh_Freq") == 'monthly'),  f.datediff(f.current_date(), f.date_sub(f.add_months(f.col("Latest_Data_Partition_Available"),1),1))  )) \
                       .withColumn("Is_Data_Refresh_Today", f.when(f.current_date() == f.col("Last_Data_Refresh_Date"),"Y").otherwise("N")) \
                       .withColumnRenamed("table_path", "Dataset_Path") \
                       .withColumn("Need_Supervision", f.when( (f.col("Data_Refresh_Freq") == 'daily') & (f.col("Days_Since_Last_Refresh") > 1) & (f.col("Is_Data_Refresh_Today") == 'N'), "Y").when( (f.col("Data_Refresh_Freq") == 'weekly') & (f.col("Days_Since_Last_Refresh") > 7) & (f.col("Is_Data_Refresh_Today") == 'N'), "Y").when( (f.col("Data_Refresh_Freq") == 'monthly') & (f.col("Days_Since_Last_Refresh") > 31) & (f.col("Is_Data_Refresh_Today") == 'N'), "Y").otherwise("N")  ) \
                      .withColumn("ops_report_updated_date", f.current_date())



In [7]:
display(ops_report.select("Domain_Name","Feature_Layer","Dataset_Name","Last_Data_Refresh_Date","Days_Since_Last_Refresh","Latest_Data_Partition_Available","Data_Latency","Data_Refresh_Freq","Is_Data_Refresh_Today" ,"Need_Supervision", "Dataset_Path","ops_report_updated_date"))

Domain_Name,Feature_Layer,Dataset_Name,Last_Data_Refresh_Date,Days_Since_Last_Refresh,Latest_Data_Partition_Available,Data_Latency,Data_Refresh_Freq,Is_Data_Refresh_Today,Need_Supervision,Dataset_Path,ops_report_updated_date
BILLING,l2_features,l2_billing_and_payments_weekly_popular_topup_hour_intermediate,2020-05-04,4,2020-04-20,12,weekly,N,N,/mnt/customer360-blob-output/C360/BILLING/l2_features/l2_billing_and_payments_weekly_popular_topup_hour_intermediate/,2020-05-08
BILLING,l3_features,l3_billing_and_payments_monthly_topup_and_volume,2020-04-26,12,2020-03-01,38,monthly,N,N,/mnt/customer360-blob-output/C360/BILLING/l3_features/l3_billing_and_payments_monthly_topup_and_volume/,2020-05-08
USAGE,l1_features,l1_usage_ru_a_vas_postpaid_usg_daily,2020-05-08,0,2020-04-23,15,daily,Y,N,/mnt/customer360-blob-output/C360/USAGE/l1_features/l1_usage_ru_a_vas_postpaid_usg_daily/,2020-05-08
USAGE,l1_features,l1_usage_incoming_call_relation_sum_ir_daily,2020-05-07,1,2020-04-30,8,daily,N,N,/mnt/customer360-blob-output/C360/USAGE/l1_features/l1_usage_incoming_call_relation_sum_ir_daily/,2020-05-08
BILLING,l1_features,l1_billing_and_payments_daily_time_since_last_top_up,2020-05-08,0,2020-05-01,7,daily,Y,N,/mnt/customer360-blob-output/C360/BILLING/l1_features/l1_billing_and_payments_daily_time_since_last_top_up/,2020-05-08
BILLING,l1_features,l1_billing_and_payments_daily_topup_and_volume,2020-05-08,0,2020-05-01,7,daily,Y,N,/mnt/customer360-blob-output/C360/BILLING/l1_features/l1_billing_and_payments_daily_topup_and_volume/,2020-05-08
REVENUE,l3_features,l3_revenue_postpaid_ru_f_sum_revenue_by_service_monthly,2020-05-07,1,2020-03-01,38,monthly,N,N,/mnt/customer360-blob-output/C360/REVENUE/l3_features/l3_revenue_postpaid_ru_f_sum_revenue_by_service_monthly/,2020-05-08
PRODUCT,l1_features,l1_product_active_fbb_customer_features_daily,2020-04-07,31,2020-03-23,46,daily,N,Y,/mnt/customer360-blob-output/C360/PRODUCT/l1_features/l1_product_active_fbb_customer_features_daily/,2020-05-08
REVENUE,l2_features,l2_revenue_prepaid_pru_f_usage_multi_weekly,2020-05-04,4,2020-04-20,12,weekly,N,N,/mnt/customer360-blob-output/C360/REVENUE/l2_features/l2_revenue_prepaid_pru_f_usage_multi_weekly/,2020-05-08
STREAM,l1_features,l1_streaming_visit_count_and_download_traffic_feature,2020-04-16,22,2020-04-08,30,daily,N,Y,/mnt/customer360-blob-output/C360/STREAM/l1_features/l1_streaming_visit_count_and_download_traffic_feature/,2020-05-08


In [8]:
ops_report = ops_report.select("Domain_Name","Feature_Layer","Dataset_Name","Last_Data_Refresh_Date","Days_Since_Last_Refresh","Latest_Data_Partition_Available","Data_Latency","Data_Refresh_Freq","Is_Data_Refresh_Today" ,"Need_Supervision", "Dataset_Path","ops_report_updated_date")

ops_report.write.partitionBy("ops_report_updated_date").format("parquet").mode("append").save("/mnt/customer360-blob-output/C360/ops_report/")

In [9]:
display(spark.read.parquet('/mnt/customer360-blob-output/C360/ops_report/'))

Domain_Name,Feature_Layer,Dataset_Name,Last_Data_Refresh_Date,Days_Since_Last_Refresh,Latest_Data_Partition_Available,Data_Latency,Data_Refresh_Freq,Is_Data_Refresh_Today,Need_Supervision,Dataset_Path,ops_report_updated_date
BILLING,l2_features,l2_billing_and_payments_weekly_popular_topup_day_intermediate,2020-05-04,4,2020-04-20,12,weekly,N,N,/mnt/customer360-blob-output/C360/BILLING/l2_features/l2_billing_and_payments_weekly_popular_topup_day_intermediate/,2020-05-08
BILLING,l2_features,l2_billing_and_payments_weekly_most_popular_top_up_channel,2020-05-04,4,2020-04-20,12,weekly,N,N,/mnt/customer360-blob-output/C360/BILLING/l2_features/l2_billing_and_payments_weekly_most_popular_top_up_channel/,2020-05-08
REVENUE,l3_features,l3_revenue_postpaid_ru_f_sum_revenue_by_service_monthly,2020-05-07,1,2020-03-01,38,monthly,N,N,/mnt/customer360-blob-output/C360/REVENUE/l3_features/l3_revenue_postpaid_ru_f_sum_revenue_by_service_monthly/,2020-05-08
REVENUE,l3_features,l3_revenue_prepaid_ru_f_sum_revenue_by_service_monthly,2020-05-07,1,2020-03-01,38,monthly,N,N,/mnt/customer360-blob-output/C360/REVENUE/l3_features/l3_revenue_prepaid_ru_f_sum_revenue_by_service_monthly/,2020-05-08
BILLING,l2_features,l2_billing_and_payments_weekly_time_since_last_top_up,2020-05-04,4,2020-04-20,12,weekly,N,N,/mnt/customer360-blob-output/C360/BILLING/l2_features/l2_billing_and_payments_weekly_time_since_last_top_up/,2020-05-08
STREAM,l1_features,l1_streaming_visit_count_and_download_traffic_feature,2020-04-16,22,2020-04-08,30,daily,N,Y,/mnt/customer360-blob-output/C360/STREAM/l1_features/l1_streaming_visit_count_and_download_traffic_feature/,2020-05-08
BILLING,l1_features,l1_billing_and_payments_daily_time_since_last_top_up,2020-05-08,0,2020-05-01,7,daily,Y,N,/mnt/customer360-blob-output/C360/BILLING/l1_features/l1_billing_and_payments_daily_time_since_last_top_up/,2020-05-08
BILLING,l1_features,l1_billing_and_payments_daily_before_top_up_balance,2020-05-08,0,2020-05-02,6,daily,Y,N,/mnt/customer360-blob-output/C360/BILLING/l1_features/l1_billing_and_payments_daily_before_top_up_balance/,2020-05-08
BILLING,l2_features,l2_billing_and_payments_weekly_last_top_up_channel,2020-05-04,4,2020-04-20,12,weekly,N,N,/mnt/customer360-blob-output/C360/BILLING/l2_features/l2_billing_and_payments_weekly_last_top_up_channel/,2020-05-08
BILLING,l2_features,l2_billing_and_payments_weekly_popular_topup_day,2020-05-04,4,2020-04-20,12,weekly,N,N,/mnt/customer360-blob-output/C360/BILLING/l2_features/l2_billing_and_payments_weekly_popular_topup_day/,2020-05-08
